<a href="https://colab.research.google.com/github/ruitenbeek/thesis/blob/main/1step_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/thesis/code')
!pwd

/content/gdrive/My Drive/thesis/code


In [ ]:
from sklearn.dummy import DummyClassifier
from csv import DictReader
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
def read_file(file):
    data = list()
    abu_count = 0
    off_count = 0
    not_count = 0
    with open(file, 'r') as f:
        reader = DictReader(f, delimiter='\t')
        for row in reader:
            if (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and (row['explicitness'] == 'IMPLICIT' or row['explicitness'] == 'EXPLICIT'):
                data.append([row['text'], 'OFF'])
                off_count += 1
            elif row['abusive'] == 'IMPLICIT' or row['abusive'] == 'EXPLICIT':
                data.append([row['text'], 'ABU'])
                abu_count += 1
            elif (row['abusive'] == 'NOT' or row['abusive'] == 'UNKNOWN') and row['explicitness'] == 'NOT':
                data.append([row['text'], 'NOT'])
                not_count += 1
    print(f'ABU: %i\nOFF: %i\nNOT: %i' % (abu_count, off_count, not_count))
    data_df = pd.DataFrame(data)
    data_df.columns = ['text', 'label']
    return data_df

In [ ]:
def split_labels(data_df):
    data_X = data_df.text.tolist()
    data_y = data_df.label.tolist()
    return data_X, data_y

In [ ]:
def dummy(train_X, train_y):
    model = DummyClassifier(strategy='most_frequent')
    model.fit(train_X, train_y)
    return model

In [ ]:
def evaluation(model, test_X, test_y):
    pred_y = model.predict(test_X)
    target_names = ['ABU', 'NOT', 'OFF']
    print(classification_report(test_y, pred_y, target_names=target_names, digits=2))

In [ ]:
print('###TRAIN Split###')
train_data = read_file('train_final_pp.csv')
print('\n###TEST Split###')
test_data = read_file('test_final_pp.csv')
train_X, train_y = split_labels(train_data)
test_X, test_y = split_labels(test_data)

model = dummy(train_X, train_y)
print('\n####TEST Results####')
evaluation(model, test_X, test_y)

###TRAIN Split###
ABU: 1143
OFF: 1445
NOT: 5176

###TEST Split###
ABU: 637
OFF: 399
NOT: 2072

####TEST Results####
              precision    recall  f1-score   support

         ABU       0.00      0.00      0.00       637
         NOT       0.67      1.00      0.80      2072
         OFF       0.00      0.00      0.00       399

    accuracy                           0.67      3108
   macro avg       0.22      0.33      0.27      3108
weighted avg       0.44      0.67      0.53      3108



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
